In [1]:
# Copyright 2022, Gurobi Optimization, LLC

import numpy as np
import pandas as pd
import scipy.sparse as sp
import gurobipy as gp
from gurobipy import GRB

In [ ]:
S=pd.read_excel(io=r'../data/fo_result_402.xlsx',sheet_name='page_1',usecols="B:OM").values.T
L=pd.read_excel(io=r'../data/fo_result_8.xlsx',sheet_name='page_1',usecols="B:I").values.T

In [ ]:
m = gp.Model("opt")